# Quickstart QuotaClimat

![](../coverquotaclimat.png)

> Notebook python d'exploration pour fournir une base d'analyse et de visualisation pour toute l'équipe

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime

##### Charte graphique plotly

In [3]:
# CONFIG AND THEMES
COLOR_SEQUENCE = [
    "rgb(230, 50, 24)",
    "rgb(240, 73, 70)",
    "rgb(243, 127, 125)",
    "rgb(248, 182, 181)",
    "rgb(209, 220, 197)",
    "rgb(137, 168, 141)",
    "rgb(59, 111, 66)",
    "rgb(66, 66, 66)",
]

SMALL_SEQUENCE2 = [
    "rgb(230, 50, 24)",
    "rgb(59, 111, 66)",
]

WARMING_STRIPES_SEQUENCE =  ['#08306b', '#08519c', '#2171b5', '#4292c6',
    '#6baed6', '#9ecae1', '#c6dbef', '#deebf7',
    '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a',
    '#ef3b2c', '#cb181d', '#a50f15', '#67000d',
]

COLOR_SEQUENCE = COLOR_SEQUENCE #+ px.colors.qualitative.Antique
px.defaults.template = "plotly_white"
px.defaults.color_discrete_sequence = COLOR_SEQUENCE

# Create Plotly theme to set as default
THEME = go.layout.Template()
THEME.layout.treemapcolorway = COLOR_SEQUENCE
THEME.layout.sunburstcolorway = COLOR_SEQUENCE
THEME.layout.colorway = COLOR_SEQUENCE
THEME.layout.piecolorway = COLOR_SEQUENCE
THEME.layout.font = {"family":"Helvetica"}
px.defaults.template = THEME

# Visualize Colormap
fig = px.bar(
    pd.DataFrame({"color":COLOR_SEQUENCE}).assign(value = lambda x : 1,palette = lambda x : "QUOTACLIMAT").append(
    pd.DataFrame({"color":WARMING_STRIPES_SEQUENCE}).assign(value = lambda x : 1,palette = lambda x : "WARMINGSTRIPES")
    ),
    x = "value",
    y = "palette",
    color = "color",
    height = 300,
    
    color_discrete_sequence=COLOR_SEQUENCE + WARMING_STRIPES_SEQUENCE
)
fig.update_layout(xaxis={"showticklabels":False,"title":""},yaxis={"title":""},showlegend=False)
fig

C:\Users\Gregoire\AppData\Local\Temp\ipykernel_21288\3701879964.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd.DataFrame({"color":COLOR_SEQUENCE}).assign(value = lambda x : 1,palette = lambda x : "QUOTACLIMAT").append(


# Récupération des données

>  A changer plus tard une fois la base de données SQL mise en place

In [4]:
import os
os.listdir("../data/keywords")

['20221015_lastmonth_all_activiste.xlsx',
 '20221015_lastmonth_all_biodiversité.xlsx',
 '20221015_lastmonth_all_changement climatique.xlsx',
 '20221015_lastmonth_all_décroissance.xlsx',
 '20221015_lastmonth_all_GIEC.xlsx',
 '20221015_lastmonth_all_réchauffement climatique.xlsx',
 '20221015_lastweek_all_sobriété.xlsx',
 '20221015_lastyear_bfmtv_changement climatique.xlsx',
 'sobriety']

## Lecture d'un extrait Mediatree

In [5]:
data = pd.read_excel('../data/keywords/20221015_lastmonth_all_changement climatique.xlsx')
data.head()

,CHANNEL,RADIO,DATE,TEXT,HIGHLIGHT,START CHUNK,END CHUNK,ORIGIN,URL
0,tv-vendee,False,2022-10-06T06-58-00,<unk> <unk> <unk> la fête de la science s' inv...,du onze au seize octobre profitez d' événemen...,2022-10-06T06-58-00,2022-10-06T07-00-00,s2t,https://keywords.mediatree.fr/player/?fifo=tv-...
1,tv78-edd,False,2022-10-07T13-50-00,en ressources augmentation de la population mo...,qu' il est urgent de valoriser lors de la tra...,2022-10-07T13-50-00,2022-10-07T13-52-00,s2t,https://keywords.mediatree.fr/player/?fifo=tv7...
2,rcf-nievre-edd,True,2022-10-11T18-12-00,d' adaptation au changement climatique le grac...,d' adaptation au changement climatique le grac...,2022-10-11T18-12-00,2022-10-11T18-14-00,s2t,https://keywords.mediatree.fr/player/?fifo=rcf...
3,fbleu-vaucluse,True,2022-09-29T07-14-00,notre alimentation oui et c' est le thème de l...,du changement climatique c' est bien ça oui e...,2022-09-29T07-14-00,2022-09-29T07-16-00,s2t,https://keywords.mediatree.fr/player/?fifo=fbl...
4,tebeo,False,2022-10-09T14-32-00,<unk> les groupes scolaires delestre organisen...,valait le détour <unk> <unk> <unk> <unk> le c...,2022-10-09T14-32-00,2022-10-09T14-34-00,s2t,https://keywords.mediatree.fr/player/?fifo=teb...


In [6]:
channels = pd.read_excel("../data/channels.xlsx")
channels.head()

,CHANNEL,CHANNEL_NAME
0,tf1,TF1
1,france2,France 2
2,france3,France 3
3,cplus,Canal+
4,france5,France 5


## Préparation des données

In [7]:
def process_mediatree_extract(path_file,path_channels):
    data = pd.read_excel(path_file)
    channels = pd.read_excel(path_channels)
    
    data = (
        data
        .merge(channels,on = "CHANNEL")
        .rename(columns = {"START CHUNK" : "START_CHUNK","END CHUNK":"END_CHUNK"})
        .assign(DATE = lambda x : pd.to_datetime(x["DATE"],format = "%Y-%m-%dT%H-%M-%S"))
        .assign(TIME = lambda x : x["DATE"].dt.time)
        .assign(TIME = lambda x : x["TIME"].map(lambda y : datetime.timedelta(hours=y.hour, minutes=y.minute, seconds=y.second)))
        .assign(MEDIA = lambda x : x["RADIO"].map(lambda y : "Radio" if y else "TV"))
        .assign(FILENAME = lambda x : path_file)
        .assign(COUNT = lambda x : 1)
        .assign(DURATION = lambda x : 2)
        .assign(KEYWORD = lambda x : x["FILENAME"].map(lambda y : y.rsplit("_",1)[-1].replace(".xlsx","")))
        .drop(columns = ["ORIGIN","START_CHUNK","END_CHUNK"])
        
    )

    return data
    
data = process_mediatree_extract('../data/keywords/20221015_lastmonth_all_changement climatique.xlsx',"../data/channels.xlsx")
data.head()

,CHANNEL,RADIO,DATE,TEXT,HIGHLIGHT,URL,CHANNEL_NAME,TIME,MEDIA,FILENAME,COUNT,DURATION,KEYWORD
0,tv-vendee,False,2022-10-06 06:58:00,<unk> <unk> <unk> la fête de la science s' inv...,du onze au seize octobre profitez d' événemen...,https://keywords.mediatree.fr/player/?fifo=tv-...,TV Vendée,0 days 06:58:00,TV,../data/keywords/20221015_lastmonth_all_change...,1,2,changement climatique
1,tv-vendee,False,2022-10-07 06:58:00,<unk> la fête de la science s' invite sur le p...,octobre profitez d' événements autour du chan...,https://keywords.mediatree.fr/player/?fifo=tv-...,TV Vendée,0 days 06:58:00,TV,../data/keywords/20221015_lastmonth_all_change...,1,2,changement climatique
2,tv-vendee,False,2022-10-04 22:14:00,<unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk...,événements autour du changement climatique au...,https://keywords.mediatree.fr/player/?fifo=tv-...,TV Vendée,0 days 22:14:00,TV,../data/keywords/20221015_lastmonth_all_change...,1,2,changement climatique
3,tv-vendee,False,2022-10-04 11:56:00,ici-là tentez votre chance vous avez jusqu' à ...,de poux rouges du onze au seize octobre profi...,https://keywords.mediatree.fr/player/?fifo=tv-...,TV Vendée,0 days 11:56:00,TV,../data/keywords/20221015_lastmonth_all_change...,1,2,changement climatique
4,tv-vendee,False,2022-10-04 23:42:00,fin de cette édition merci de l' avoir suivie ...,du changement climatique au programme spectac...,https://keywords.mediatree.fr/player/?fifo=tv-...,TV Vendée,0 days 23:42:00,TV,../data/keywords/20221015_lastmonth_all_change...,1,2,changement climatique


# Analyse sur un extrait keyword Mediatree 

In [8]:
top_channels = channels.head(25)["CHANNEL_NAME"].tolist()
top_channels_tv = top_channels[:8]
top_channels

['TF1',
 'France 2',
 'France 3',
 'Canal+',
 'France 5',
 'M6',
 'BFMTV',
 'CNEWS',
 'LCI',
 'Arte',
 'Europe 1',
 'RMC',
 'RTL',
 'France Inter',
 'Euronews',
 'LCP',
 'France Info',
 'France 24',
 'France S',
 'TV5Monde',
 'BFM Business Radio',
 'BFM Business',
 'France Culture',
 'Radio Classique',
 'RFI']

## Split des chaînes

In [9]:
count = data.groupby(["CHANNEL_NAME","MEDIA"],as_index = False)["COUNT"].sum().sort_values("COUNT",ascending = False).head(30)

fig = px.bar(
    count,
    x = "CHANNEL_NAME",
    y = "COUNT",
    color = "MEDIA",
    color_discrete_sequence=SMALL_SEQUENCE2,
    text_auto = ".2s",
    category_orders={"CHANNEL_NAME": count["CHANNEL_NAME"].tolist()},
    height = 500,
    title = "Nombre de mentions par chaîne"
)

fig.update_xaxes(tickangle=-45,title=None)
fig.update_yaxes(title=None)
fig.update_layout(margin={"b":100})
fig.show()

In [10]:
count = (data
        .loc[data["CHANNEL_NAME"].isin(top_channels)]
        .groupby(["CHANNEL_NAME","MEDIA"],as_index = False)["COUNT"].sum().sort_values("COUNT",ascending = False)
)

fig = px.bar(
    count,
    x = "CHANNEL_NAME",
    y = "COUNT",
    color = "MEDIA",
    color_discrete_sequence=SMALL_SEQUENCE2,
    text_auto = ".2s",
    category_orders={"CHANNEL_NAME": count["CHANNEL_NAME"].tolist()},
    height = 500,
    title = "Nombre de mentions par chaîne"
)

fig.update_xaxes(tickangle=-45,title=None)
fig.update_yaxes(title=None)
fig.update_layout(margin={"b":100})
fig.show()

In [11]:
count = data.groupby(["MEDIA"],as_index = False)["COUNT"].sum().sort_values("COUNT",ascending = False)
count

fig = px.pie(
    count,
    names = "MEDIA",
    values = "COUNT",
    color_discrete_sequence=SMALL_SEQUENCE2,
    title = "Split TV / Radio"
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Evolution au cours du temps

In [12]:
freq = "D"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq)],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.bar(count,x = "DATE",y = "COUNT",title = "Evolution du nombre de mention au cours du temps",height = 400)
fig.show()

In [13]:
freq = "D"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"MEDIA"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.bar(count,
             x = "DATE",y = "COUNT",color = "MEDIA",
             title = "Evolution du nombre de mention au cours du temps par type de média",
             height = 400,color_discrete_sequence=SMALL_SEQUENCE2
)
fig.show()

In [14]:
freq = "D"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"MEDIA"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.area(count,
             x = "DATE",y = "COUNT",color = "MEDIA",
             title = "Evolution du nombre de mention au cours du temps par type de média",height = 400,color_discrete_sequence=SMALL_SEQUENCE2
)
fig.show()

In [15]:
freq = "D"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"MEDIA"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.area(count,
             x = "DATE",y = "COUNT",color = "MEDIA",groupnorm='fraction',
             title = "Evolution du nombre de mention au cours du temps par type de média en %",height = 400,color_discrete_sequence=SMALL_SEQUENCE2,
)
fig.update_layout(yaxis_tickformat='0%') 
fig.show()

In [16]:
freq = "6H"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"MEDIA"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.area(count,
             x = "DATE",y = "COUNT",color = "MEDIA",
             title = "Evolution du nombre de mention au cours du temps par type de média",height = 400,color_discrete_sequence=SMALL_SEQUENCE2
)
fig.show()

In [17]:
freq = "D"

count = (
    data.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"MEDIA","CHANNEL_NAME"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels_tv)]

fig = px.line(count,
             x = "DATE",y = "COUNT",color = "CHANNEL_NAME",
             title = "Evolution du nombre de mention au cours du temps par chaîne TV",height = 400
)
fig.show()

fig = px.bar(count,
             x = "DATE",y = "COUNT",color = "CHANNEL_NAME",
             title = "Evolution du nombre de mention au cours du temps par chaîne TV",height = 400
)
fig.show()

## Heures de la journée

In [18]:
freq = "1H"

count = (
    data
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq)],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : str(y)[7:12]))
)

fig = px.bar(
    count,
    x = "TIME",y = "COUNT",
)
fig.show()

In [19]:
count["TIME"].unique()

array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00',
       '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00',
       '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00',
       '21:00', '22:00', '23:00'], dtype=object)

In [20]:
freq = "1H"

count = (
    data
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq),"MEDIA"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : str(y)[7:12]))
    .sort_values("TIME",ascending = True)
)

fig = px.bar(
    count,
    text_auto = "s",
    x = "TIME",y = "COUNT",color="MEDIA",
    color_discrete_sequence=SMALL_SEQUENCE2,
    height = 400,
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée"
)
fig.show()


fig = px.area(
    count,
    x = "TIME",y = "COUNT",color = "MEDIA",groupnorm='fraction',
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée en %",height = 400,color_discrete_sequence=SMALL_SEQUENCE2,
)
fig.update_layout(yaxis_tickformat='0%') 
fig.show()

In [21]:
freq = "1H"

count = (
    data
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq),"CHANNEL_NAME"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : str(y)[7:12]))
    .sort_values("TIME",ascending = True)
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels_tv)]

fig = px.bar(
    count,
    x = "TIME",y = "COUNT",color="CHANNEL_NAME",
    height = 400,
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée"
)
fig.show()


fig = px.area(
    count,
    x = "TIME",y = "COUNT",color = "CHANNEL_NAME",groupnorm='fraction',
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée en %",height = 400,
)
fig.update_layout(yaxis_tickformat='0%') 
fig.show()

### Par chaîne

In [22]:
freq = "3H"

count = (
    data
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq),"CHANNEL_NAME"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : str(y)[7:12]))
    .sort_values("TIME",ascending = True)
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels_tv)]

fig = px.bar(
    count,
    x = "TIME",y = "COUNT",color="CHANNEL_NAME",
    height = 400,
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée"
)
fig.show()


fig = px.area(
    count,
    x = "TIME",y = "COUNT",color = "CHANNEL_NAME",groupnorm='fraction',
    category_orders={"TIME":count["TIME"].unique()},
    title = "Répartition des mentions par heure de la journée en %",height = 400,
)
fig.update_layout(yaxis_tickformat='0%') 
fig.show()

In [23]:
count

,TIME,CHANNEL_NAME,COUNT
2,00:00,BFMTV,2
4,00:00,France 2,2
25,03:00,Canal+,1
24,03:00,BFMTV,2
81,06:00,France 2,18
66,06:00,CNEWS,1
60,06:00,BFMTV,5
491,09:00,TF1,7
342,09:00,France 2,5
331,09:00,CNEWS,5


In [24]:
freq = "4H"

def parse_period(y,freq):
    hours = int(str(y)[7:9])
    return f"{hours}-{hours+int(freq.replace('H',''))}h"

count = (
    data
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq),"CHANNEL_NAME"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : parse_period(y,freq)))
    .sort_values("TIME",ascending = True)
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels)]

fig = px.treemap(
    count,
    path = ["CHANNEL_NAME","TIME"],
    values = "COUNT",
)
fig

# Comparer deux fichiers

In [25]:
data_climat = process_mediatree_extract('../data/keywords/20221015_lastmonth_all_changement climatique.xlsx',"../data/channels.xlsx")
data_biodiv = process_mediatree_extract('../data/keywords/20221015_lastmonth_all_biodiversité.xlsx',"../data/channels.xlsx")
data_agg = pd.concat([data_climat,data_biodiv],axis = 0,ignore_index = True)

In [26]:
freq = "D"

count = (
    data_agg.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"KEYWORD"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.bar(count,
             x = "DATE",y = "COUNT",color = "KEYWORD",
             title = "Evolution du nombre de mention au cours du temps par mot clé",
             height = 400,color_discrete_sequence=SMALL_SEQUENCE2
)
fig.show()

In [27]:
freq = "D"

count = (
    data_agg.set_index(["DATE"])
    .groupby([pd.Grouper(freq = freq),"KEYWORD"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
)

fig = px.area(count,
             x = "DATE",y = "COUNT",color = "KEYWORD",groupnorm='fraction',
             title = "Evolution du nombre de mention au cours du temps par keyword en %",height = 400,color_discrete_sequence=SMALL_SEQUENCE2,
)
fig.update_layout(yaxis_tickformat='0%') 
fig.show()

In [28]:

count = (
    data_agg
    .groupby(["CHANNEL_NAME","MEDIA","KEYWORD"],as_index = False)
    ["COUNT"].sum()
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels_tv)]

fig = px.bar(
    count,
    x = "CHANNEL_NAME",y = "COUNT",color="KEYWORD",
    height = 400,
    title = "Répartition des mentions par mot clé par chaîne TV principale",
    color_discrete_sequence=SMALL_SEQUENCE2,
    text_auto = "s",
)
fig.show()

In [29]:
freq = "4H"

def parse_period(y,freq):
    hours = int(str(y)[7:9])
    return f"{hours}-{hours+int(freq.replace('H',''))}h"

count = (
    data_agg
    .set_index(["TIME"])
    .groupby([pd.Grouper(freq = freq),"CHANNEL_NAME","KEYWORD"],as_index = True)
    ["COUNT"].sum()
    .reset_index()
    .assign(TIME = lambda x: x["TIME"].map(lambda y : parse_period(y,freq)))
    .sort_values("TIME",ascending = True)
)

count = count.loc[count["CHANNEL_NAME"].isin(top_channels)]

fig = px.treemap(
    count,
    path = ["CHANNEL_NAME","KEYWORD","TIME"],
    values = "COUNT",
    height = 800
)
fig